# library

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from tqdm.auto import tqdm

import torchvision.models.vgg as vgg

# hyperparameter

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# dataset

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
train_dataset = torchvision.datasets.CIFAR10(root = './cifar10', train= True,
                                             download = True, transform = transform)

test_dataset = torchvision.datasets.CIFAR10(root = './cifar10', train = False,
                                            download = True, transform = transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 512, shuffle = True, num_workers = 0)

100%|██████████| 170498071/170498071 [00:13<00:00, 12984311.41it/s]


Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified


In [5]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# model

In [6]:
cfg = [32, 32, 'M', 64, 64, 128, 128, 128, 'M', 256, 256, 256, 512, 512, 512, 'M']

In [7]:
class VGG(nn.Module):

    def __init__(self, features, num_classes = 1000, init_weights = True):

        super(VGG, self).__init__()

        self.features = features
        #self.avgpool = nn.AdaptiveAvgPool2d((7,7))

        self.classifier = nn.Sequential(
            nn.Linear(512 * 4 * 4, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

        if init_weights:

            self._initialize_weights()

    def forward(self, x):

        x = self.features(x)
        #x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x

    def _initialize_weights(self):

        for m in self.modules():

            if isinstance(m, nn.Conv2d):

                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity = 'relu')

                if m.bias is not None:

                    nn.init.constant_(m.bias, 0)

            elif isinstance(m, nn.BatchNorm2d):

                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

            elif isinstance(m, nn.Linear):

                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


In [8]:
vgg16 = VGG(vgg.make_layers(cfg), 10, True).to(device)

In [9]:
# test

a = torch.Tensor(1,3,32,32).to(device)
out = vgg16(a)
print(out)

tensor([[ 173835.7812, -217598.6406,    1402.0488,  -59023.4102,  112366.3125,
         -388747.0625,  -36495.2891,   90450.8672,  310690.5938, -374813.8750]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


# training

In [10]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(vgg16.parameters(), lr = 0.005, momentum = 0.9)

#stepsize로 정한 것 마다 epoch 돌때 원래 learning rate에 gamma를 곱하여 새로운 learning rate로 한 뒤 epoch을 돌게함
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.9)

In [11]:
vgg16.train()

epochs = 50

for epoch in range(epochs):

    running_loss = 0.0
    lr_sche.step()

    for i,data in enumerate(train_loader, 0):

        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = vgg16(inputs)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 30 == 29:

            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 30))

            running_loss = 0.0

print('finished training')

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


[1,    30] loss: 2.302
[1,    60] loss: 2.297
[1,    90] loss: 2.276
[2,    30] loss: 2.172
[2,    60] loss: 2.090
[2,    90] loss: 1.990
[3,    30] loss: 1.879
[3,    60] loss: 1.791
[3,    90] loss: 1.739
[4,    30] loss: 1.668
[4,    60] loss: 1.600
[4,    90] loss: 1.566
[5,    30] loss: 1.498
[5,    60] loss: 1.479
[5,    90] loss: 1.420
[6,    30] loss: 1.395
[6,    60] loss: 1.381
[6,    90] loss: 1.323
[7,    30] loss: 1.289
[7,    60] loss: 1.274
[7,    90] loss: 1.236
[8,    30] loss: 1.193
[8,    60] loss: 1.183
[8,    90] loss: 1.132
[9,    30] loss: 1.105
[9,    60] loss: 1.074
[9,    90] loss: 1.062
[10,    30] loss: 1.020
[10,    60] loss: 1.014
[10,    90] loss: 1.071
[11,    30] loss: 0.970
[11,    60] loss: 0.969
[11,    90] loss: 0.957
[12,    30] loss: 0.903
[12,    60] loss: 0.918
[12,    90] loss: 0.919
[13,    30] loss: 0.875
[13,    60] loss: 0.875
[13,    90] loss: 0.849
[14,    30] loss: 0.809
[14,    60] loss: 0.823
[14,    90] loss: 0.828
[15,    30] loss: 0

# test

In [14]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 512, shuffle = False, num_workers = 0)

data_iter = iter(test_loader)

#https://dacon.io/forum/407531
images, labels = next(data_iter)

print('groundTruth:', ' '.join('%5s'%classes[labels[j]] for j in range(4)))

groundTruth:   cat  ship  ship plane


In [15]:
vgg16.eval()

outputs = vgg16(images.to(device))

_, predicted = torch.max(outputs, 1)

print('predicted:', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

predicted:   cat  ship  ship plane


In [17]:
correct = 0
total = 0

vgg16.eval()

with torch.no_grad():

    for data in tqdm(test_loader):

        images, labels = data

        images = images.to(device)
        labels = labels.to(device)
        outputs = vgg16(images)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)

        correct += (predicted == labels).sum().item()

print('accuracy of the network on the 10000 test images: %d %%' %(
    100*correct/total
))

  0%|          | 0/20 [00:00<?, ?it/s]

accuracy of the network on the 10000 test images: 78 %
